<a href="https://colab.research.google.com/github/Alfaalputra/Alfaalputra/blob/main/FP-Growth_Algorithm_Association_Rules_TBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Preprocessing Data

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/dataset TB Anak/Tuberkulosis.csv')
dataset.head()

,Timestamp,Tanggal Lahir,Umur,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Alamat (mohon sertakan nama kelurahan dan kecamatan),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah atau sedang dalam pengobatan tuberkulosis?,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?
0,07/10/2021 12:03:36,20/11/2014,6 Tahun,125.0,19.0,Laki - laki,Noeha,Nuha,Sorowako,"Kec. Nuha, Sorowako",Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Ya,Tidak,Ya,Ya,Diare,Ya,Tidak,Tidak,Ya,Hipertensi,Tidak,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi
1,07/10/2021 14:48:47,18/04/2011,10 Tahun,120.0,30.0,Perempuan,Parang Tambung,Tamalate,Makassar,Dg.tata 1 perumahan tata satu indah blok 1b,Wiraswasta,ASN,2.000.000 - 5.000.000,Ya,Tidak,Tidak,Ya,Brongkhitis,Ya,Tidak,Tidak,Ya,Tidak Ada,Tidak,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi
2,10/10/2021 21:54:49,04/05/2018,3 Tahun 5 Bulan,104.0,17.0,Laki - laki,Kalebajeng,Bajeng,Gowa,Perum bajeng permai blok u no 4. Kel kalebajen...,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Ya,Tidak,Ya,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi
3,11/10/2021 17:03:37,20/05/2013,8 Tahun,120.0,23.0,Perempuan,Bone,Masamba,Luwu Utara,"Jl.Mujahidin Kel.Bone Kec.Masamba,Luwu Utara",ASN,ASN,5.000.000 - 10.000.000,Ya,Tidak,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi
4,14/10/2021 16:39:34,11/02/2019,2 Tahun 8 Bulan,90.0,14.0,Perempuan,Borong,Manggala,Makassar,"Perm. Graha Indah Family, Kel. Borong, Kec. Ma...",ASN,Dokter Gigi,> 10.000.000,Ya,Tidak,Ya,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Ya,Bronkitis,Tidak,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup


In [ ]:
cp = dataset.copy()
cp

,Timestamp,Tanggal Lahir,Umur,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Alamat (mohon sertakan nama kelurahan dan kecamatan),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah atau sedang dalam pengobatan tuberkulosis?,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?
0,07/10/2021 12:03:36,20/11/2014,6 Tahun,125.0,19.0,Laki - laki,Noeha,Nuha,Sorowako,"Kec. Nuha, Sorowako",Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Ya,Tidak,Ya,Ya,Diare,Ya,Tidak,Tidak,Ya,Hipertensi,Tidak,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi
1,07/10/2021 14:48:47,18/04/2011,10 Tahun,120.0,30.0,Perempuan,Parang Tambung,Tamalate,Makassar,Dg.tata 1 perumahan tata satu indah blok 1b,Wiraswasta,ASN,2.000.000 - 5.000.000,Ya,Tidak,Tidak,Ya,Brongkhitis,Ya,Tidak,Tidak,Ya,Tidak Ada,Tidak,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi
2,10/10/2021 21:54:49,04/05/2018,3 Tahun 5 Bulan,104.0,17.0,Laki - laki,Kalebajeng,Bajeng,Gowa,Perum bajeng permai blok u no 4. Kel kalebajen...,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Ya,Tidak,Ya,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi
3,11/10/2021 17:03:37,20/05/2013,8 Tahun,120.0,23.0,Perempuan,Bone,Masamba,Luwu Utara,"Jl.Mujahidin Kel.Bone Kec.Masamba,Luwu Utara",ASN,ASN,5.000.000 - 10.000.000,Ya,Tidak,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi
4,14/10/2021 16:39:34,11/02/2019,2 Tahun 8 Bulan,90.0,14.0,Perempuan,Borong,Manggala,Makassar,"Perm. Graha Indah Family, Kel. Borong, Kec. Ma...",ASN,Dokter Gigi,> 10.000.000,Ya,Tidak,Ya,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Ya,Bronkitis,Tidak,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,29/12/2021 06:10:10,06/10/2016,5 tahun,110.0,16.0,Laki - laki,Tello Baru,Panakkukang,Makassar,Jalan paccinang raya no 7 tello baru,Karyawan Swasta,PNS,5.000.001 - 10.000.000,Ya,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,54 - 120 m^2,3,4,Ada ventilasi namun kadang terbuka/tertutup
67,29/12/2021 06:12:12,21/01/2019,2 tahun,86.0,12.4,Laki - laki,Batua,Manggala,Makassar,Jalan swadaya mas no 32,Karyawan Swasta,PNS,5.000.001 - 10.000.000,Ya,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,36 - 54 m^2,2,5,Ada ventilasi namun kadang terbuka/tertutup
68,29/12/2021 06:14:40,09/07/2014,7 tahun,128.0,26.0,Perempuan,Kapasa,Tamalanrea,Makassar,Btn angkatan laut blok A2/14,PNS,PNS,5.000.001 - 10.000.000,Ya,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,54 - 120 m^2,4,6,Ada ventilasi namun kadang terbuka/tertutup
69,29/12/2021 06:17:08,09/02/2020,1 tahun,78.0,9.3,Laki - laki,Kalabajeng,Bajeng,Gowa,Bajeng permai Blok U/4,Wiraswasta,Ibu Rumah tangga,2.500.000 - 5.000.000,Ya,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,36 - 54 m^2,2,4,Ada ventilasi namun kadang terbuka/tertutup


In [ ]:
cp.drop(['Timestamp', 'Tanggal Lahir', 'Alamat (mohon sertakan nama kelurahan dan kecamatan)',
         'Apakah anak pernah atau sedang dalam pengobatan tuberkulosis?'],
             axis = 'columns', inplace = True)
cp.head()

,Umur,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?
0,6 Tahun,125.0,19.0,Laki - laki,Noeha,Nuha,Sorowako,Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Tidak,Ya,Ya,Diare,Ya,Tidak,Tidak,Ya,Hipertensi,Tidak,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi
1,10 Tahun,120.0,30.0,Perempuan,Parang Tambung,Tamalate,Makassar,Wiraswasta,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Ya,Brongkhitis,Ya,Tidak,Tidak,Ya,Tidak Ada,Tidak,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi
2,3 Tahun 5 Bulan,104.0,17.0,Laki - laki,Kalebajeng,Bajeng,Gowa,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi
3,8 Tahun,120.0,23.0,Perempuan,Bone,Masamba,Luwu Utara,ASN,ASN,5.000.000 - 10.000.000,Tidak,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi
4,2 Tahun 8 Bulan,90.0,14.0,Perempuan,Borong,Manggala,Makassar,ASN,Dokter Gigi,> 10.000.000,Tidak,Ya,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Ya,Bronkitis,Tidak,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup


In [ ]:
new = cp['Umur'].str.split(r'(\d+)', expand = True)
new = new.replace([None], '0')

cp['Tahun']= new[1]
cp['Bulan'] = new[3]

cp.drop(['Umur'], axis = 'columns', inplace = True)
cp

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?,Tahun,Bulan
0,125.0,19.0,Laki - laki,Noeha,Nuha,Sorowako,Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Tidak,Ya,Ya,Diare,Ya,Tidak,Tidak,Ya,Hipertensi,Tidak,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi,6,0
1,120.0,30.0,Perempuan,Parang Tambung,Tamalate,Makassar,Wiraswasta,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Ya,Brongkhitis,Ya,Tidak,Tidak,Ya,Tidak Ada,Tidak,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi,10,0
2,104.0,17.0,Laki - laki,Kalebajeng,Bajeng,Gowa,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi,3,5
3,120.0,23.0,Perempuan,Bone,Masamba,Luwu Utara,ASN,ASN,5.000.000 - 10.000.000,Tidak,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi,8,0
4,90.0,14.0,Perempuan,Borong,Manggala,Makassar,ASN,Dokter Gigi,> 10.000.000,Tidak,Ya,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Ya,Bronkitis,Tidak,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,110.0,16.0,Laki - laki,Tello Baru,Panakkukang,Makassar,Karyawan Swasta,PNS,5.000.001 - 10.000.000,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,54 - 120 m^2,3,4,Ada ventilasi namun kadang terbuka/tertutup,5,0
67,86.0,12.4,Laki - laki,Batua,Manggala,Makassar,Karyawan Swasta,PNS,5.000.001 - 10.000.000,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,36 - 54 m^2,2,5,Ada ventilasi namun kadang terbuka/tertutup,2,0
68,128.0,26.0,Perempuan,Kapasa,Tamalanrea,Makassar,PNS,PNS,5.000.001 - 10.000.000,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,54 - 120 m^2,4,6,Ada ventilasi namun kadang terbuka/tertutup,7,0
69,78.0,9.3,Laki - laki,Kalabajeng,Bajeng,Gowa,Wiraswasta,Ibu Rumah tangga,2.500.000 - 5.000.000,Tidak,Tidak,Tidak,NaN,Tidak,Tidak,Tidak,Tidak,NaN,Tidak,36 - 54 m^2,2,4,Ada ventilasi namun kadang terbuka/tertutup,1,0


In [ ]:
cp.replace(np.nan, 'Tidak Ada', inplace = True)
cp

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?,Tahun,Bulan
0,125.0,19.0,Laki - laki,Noeha,Nuha,Sorowako,Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Tidak,Ya,Ya,Diare,Ya,Tidak,Tidak,Ya,Hipertensi,Tidak,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi,6,0
1,120.0,30.0,Perempuan,Parang Tambung,Tamalate,Makassar,Wiraswasta,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Ya,Brongkhitis,Ya,Tidak,Tidak,Ya,Tidak Ada,Tidak,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi,10,0
2,104.0,17.0,Laki - laki,Kalebajeng,Bajeng,Gowa,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi,3,5
3,120.0,23.0,Perempuan,Bone,Masamba,Luwu Utara,ASN,ASN,5.000.000 - 10.000.000,Tidak,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi,8,0
4,90.0,14.0,Perempuan,Borong,Manggala,Makassar,ASN,Dokter Gigi,> 10.000.000,Tidak,Ya,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Ya,Bronkitis,Tidak,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,110.0,16.0,Laki - laki,Tello Baru,Panakkukang,Makassar,Karyawan Swasta,PNS,5.000.001 - 10.000.000,Tidak,Tidak,Tidak,Tidak Ada,Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,3,4,Ada ventilasi namun kadang terbuka/tertutup,5,0
67,86.0,12.4,Laki - laki,Batua,Manggala,Makassar,Karyawan Swasta,PNS,5.000.001 - 10.000.000,Tidak,Tidak,Tidak,Tidak Ada,Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,36 - 54 m^2,2,5,Ada ventilasi namun kadang terbuka/tertutup,2,0
68,128.0,26.0,Perempuan,Kapasa,Tamalanrea,Makassar,PNS,PNS,5.000.001 - 10.000.000,Tidak,Tidak,Tidak,Tidak Ada,Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4,6,Ada ventilasi namun kadang terbuka/tertutup,7,0
69,78.0,9.3,Laki - laki,Kalabajeng,Bajeng,Gowa,Wiraswasta,Ibu Rumah tangga,2.500.000 - 5.000.000,Tidak,Tidak,Tidak,Tidak Ada,Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,36 - 54 m^2,2,4,Ada ventilasi namun kadang terbuka/tertutup,1,0


In [ ]:
cp.dtypes

Tinggi badan (dalam cm)                                                                                                                                     float64
Berat badan (dalam kg)                                                                                                                                      float64
Jenis Kelamin                                                                                                                                                object
Alamat (Kelurahan)                                                                                                                                           object
Alamat (Kecamatan)                                                                                                                                           object
Alamat (Kab/Kota)                                                                                                                                            object
Pekerjaan Ayah  

In [ ]:
cp['Berapa jumlah kamar tidur dalam rumah?'] = cp['Berapa jumlah kamar tidur dalam rumah?'].astype(str)
cp['Berapa jumlah orang yang tinggal dalam satu rumah?'] = cp['Berapa jumlah orang yang tinggal dalam satu rumah?'].astype(str)

In [ ]:
cp['Alamat (Kelurahan)'] = 'Kelurahan ' + cp['Alamat (Kelurahan)']
cp['Alamat (Kecamatan)'] = 'Kecamatan ' + cp['Alamat (Kecamatan)']
cp['Pekerjaan Ayah'] = 'Ayah ' + cp['Pekerjaan Ayah']
cp['Pekerjaan Ibu'] = 'Ibu ' + cp['Pekerjaan Ibu']
cp['Berapa jumlah kamar tidur dalam rumah?'] = cp['Berapa jumlah kamar tidur dalam rumah?'] + ' Kamar'
cp['Berapa jumlah orang yang tinggal dalam satu rumah?'] = cp['Berapa jumlah orang yang tinggal dalam satu rumah?'] + ' Orang'
cp.head()

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?,Tahun,Bulan
0,125.0,19.0,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak,Ya,Ya,Diare,Ya,Tidak,Tidak,Ya,Hipertensi,Tidak,> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,6,0
1,120.0,30.0,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak,Tidak,Ya,Brongkhitis,Ya,Tidak,Tidak,Ya,Tidak Ada,Tidak,> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,10,0
2,104.0,17.0,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,3,5
3,120.0,23.0,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,8,0
4,90.0,14.0,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak,Ya,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Ya,Bronkitis,Tidak,54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,2,8


In [ ]:
cp['Apakah anak pernah mengalami penyakit diabetes?'] = cp['Apakah anak pernah mengalami penyakit diabetes?'] + ' pernah diabetes'
cp['Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?'] = cp['Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?'] + ' Telah BCG'
cp['Apakah anak pernah di opname sebelumnya?'] = cp['Apakah anak pernah di opname sebelumnya?'] + ' pernah Opname'
cp['Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan)'] = cp['Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan)'] + ' ASI Ekslusif'
cp['Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?'] = cp['Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?'] + ' ada riwayat'
cp['Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?'] = cp['Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?'] + ' ada riwayat penyakit diabetes (orang tua)'
cp['Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?'] = cp['Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?'] + ' ada yang mengkomsunsi obat TB dalam serumah'

cp.head()

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?,Tahun,Bulan
0,125.0,19.0,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Diare,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Hipertensi,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,6,0
1,120.0,30.0,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Telah BCG,Ya pernah Opname,Brongkhitis,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,10,0
2,104.0,17.0,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,3,5
3,120.0,23.0,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Pernah,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,8,0
4,90.0,14.0,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Tuberkulosis,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Bronkitis,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,2,8


In [ ]:
tinggi_meter = cp['Tinggi badan (dalam cm)']/100
cp['IMT'] = cp['Berat badan (dalam kg)']/ (pow(tinggi_meter, 2))

cp

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?,Tahun,Bulan,IMT
0,125.0,19.0,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Diare,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Hipertensi,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,6,0,12.160000
1,120.0,30.0,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Telah BCG,Ya pernah Opname,Brongkhitis,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,10,0,20.833333
2,104.0,17.0,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,3,5,15.717456
3,120.0,23.0,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Pernah,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,8,0,15.972222
4,90.0,14.0,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Tuberkulosis,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Bronkitis,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,2,8,17.283951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,110.0,16.0,Laki - laki,Kelurahan Tello Baru,Kecamatan Panakkukang,Makassar,Ayah Karyawan Swasta,Ibu PNS,5.000.001 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Ada,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,3 Kamar,4 Orang,Ada ventilasi namun kadang terbuka/tertutup,5,0,13.223140
67,86.0,12.4,Laki - laki,Kelurahan Batua,Kecamatan Manggala,Makassar,Ayah Karyawan Swasta,Ibu PNS,5.000.001 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Ada,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada 

In [ ]:
cp['Tahun'] = cp['Tahun'].astype(int)
cp['Bulan'] = cp['Bulan'].astype(int)
cp.dtypes

Tinggi badan (dalam cm)                                                                                                                                     float64
Berat badan (dalam kg)                                                                                                                                      float64
Jenis Kelamin                                                                                                                                                object
Alamat (Kelurahan)                                                                                                                                           object
Alamat (Kecamatan)                                                                                                                                           object
Alamat (Kab/Kota)                                                                                                                                            object
Pekerjaan Ayah  

In [ ]:
IMT_laki = pd.read_csv('/content/gdrive/MyDrive/dataset TB Anak/status_gizi_laki.csv')
IMT_laki.head()

,Tahun,Bulan,-3 SD,-2 SD,-1 SD,Median,+1 SD,+2 SD,+3 SD
0,0,0,10.2,11.1,12.2,13.4,14.8,16.3,18.1
1,0,1,11.3,12.4,13.6,14.9,16.3,17.8,19.4
2,0,2,12.5,13.7,15.0,16.3,17.8,19.4,21.1
3,0,3,13.1,14.3,15.5,16.9,18.4,20.0,21.8
4,0,4,13.4,14.5,15.8,17.2,18.7,20.3,22.1


In [ ]:
IMT_perempuan = pd.read_csv('/content/gdrive/MyDrive/dataset TB Anak/status_gizi_perempuan.csv')
IMT_perempuan.head()

,Tahun,Bulan,-3 SD,-2 SD,-1 SD,Median,+1 SD,+2 SD,+3 SD
0,0,0,10.1,11.1,12.2,13.3,14.6,16.1,17.7
1,0,1,10.8,12.0,13.2,14.6,16.0,17.5,19.1
2,0,2,11.8,13.0,14.3,15.8,17.3,19.0,20.7
3,0,3,12.4,13.6,14.9,16.4,17.9,19.7,21.5
4,0,4,12.7,13.9,15.2,16.7,18.3,20.0,22.0


In [ ]:
IMT_laki.dtypes

Tahun       int64
Bulan       int64
-3 SD     float64
-2 SD     float64
-1 SD     float64
Median    float64
+1 SD     float64
+2 SD     float64
+3 SD     float64
dtype: object

In [ ]:
list_status_gizi = []

for index, row in cp.iterrows():
  if row['Jenis Kelamin'] == 'Laki - laki':
    imt = IMT_laki 

  else:
    imt = IMT_perempuan
  
  # print(row['IMT'], row['Tahun'], row['Bulan'])
  select_row = imt[(imt['Tahun'] == row['Tahun']) & (imt['Bulan'] == row['Bulan'])]
  select_row.columns = range(select_row.shape[1])

  for column in select_row.columns:
    if column > 1:
      if row['IMT'] < select_row[column].iloc[0]:
        var_imt = column - 1
        break
 
  nilai_gizi =  (row['IMT'] - select_row[var_imt - 1]) / (select_row[var_imt] - select_row[var_imt - 1])
  nilai_gizi = nilai_gizi.iloc[0]

  if select_row[0].iloc[0] + (select_row[1].iloc[0]/12) <= 5:
    if nilai_gizi < -3.0 :
      status_gizi = 'Gizi Buruk'

    elif -3.0 <= nilai_gizi <= -2.0 :
      status_gizi = 'Gizi Kurang'

    elif -2.0 < nilai_gizi <= 1.0 :
      status_gizi = 'Gizi Baik'

    elif 1.0 < nilai_gizi <= 2.0 :
      status_gizi = 'Beresiko Gizi Lebih'

    elif 2.0 < nilai_gizi <= 3.0 :
      status_gizi = 'Gizi Lebih'

    elif nilai_gizi > 3.0 :
      status_gizi = 'Obesitas'

  else:
    if  nilai_gizi < -3.0 :
      status_gizi = 'Gizi Buruk'

    elif -3.0 >= nilai_gizi <= -2.0 :
      status_gizi = 'Gizi Kurang'

    elif -2.0 < nilai_gizi <= 1.0 :
      status_gizi = 'Gizi Baik'

    elif 1.0 < nilai_gizi <= 2.0 :
      status_gizi = 'Gizi Lebih'

    elif nilai_gizi > 2.0 :
      status_gizi = 'Obesitas'

  list_status_gizi.append(status_gizi)

cp['Status Gizi'] = list_status_gizi

# print(select_row[0].iloc[0])
# print(select_row[1].iloc[0])
# print(nilai_gizi)
# print(status_gizi)
cp

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?,Tahun,Bulan,IMT,Status Gizi
0,125.0,19.0,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Diare,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Hipertensi,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,6,0,12.160000,Gizi Lebih
1,120.0,30.0,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Telah BCG,Ya pernah Opname,Brongkhitis,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,10,0,20.833333,Gizi Lebih
2,104.0,17.0,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,3,5,15.717456,Beresiko Gizi Lebih
3,120.0,23.0,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Pernah,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,8,0,15.972222,Gizi Lebih
4,90.0,14.0,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Tuberkulosis,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Bronkitis,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,2,8,17.283951,Beresiko Gizi Lebih
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,110.0,16.0,Laki - laki,Kelurahan Tello Baru,Kecamatan Panakkukang,Makassar,Ayah Karyawan Swasta,Ibu PNS,5.000.001 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Ada,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,3 Kamar,4 Orang,Ada ventilasi namun kadang terbuka/tertutup,5,0,13.223140,Beresiko Gizi Lebih
67,86.0,12.4,Laki - laki,Kelurahan Batua,Kecamatan Manggala,Makassar,Ayah Karyawan Swasta,Ibu PNS,5.000.001 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Ada,Ti

In [ ]:
cp.drop(['Tinggi badan (dalam cm)', 'Berat badan (dalam kg)', 'Tahun',
         'Bulan', 'IMT'],
             axis = 'columns', inplace = True)
cp

,Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,Apakah anak pernah mengalami penyakit diabetes?,"Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?",Apakah anak pernah di opname sebelumnya?,"Jika pernah, anak diopname karena penyakit apa saja?",Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?,Status Gizi
0,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Diare,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Hipertensi,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,Gizi Lebih
1,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Telah BCG,Ya pernah Opname,Brongkhitis,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,Gizi Lebih
2,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,Beresiko Gizi Lebih
3,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Pernah,Ya ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,Gizi Lebih
4,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak pernah diabetes,Ya Telah BCG,Ya pernah Opname,Tuberkulosis,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Ya,Bronkitis,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,Beresiko Gizi Lebih
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Laki - laki,Kelurahan Tello Baru,Kecamatan Panakkukang,Makassar,Ayah Karyawan Swasta,Ibu PNS,5.000.001 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Ada,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,54 - 120 m^2,3 Kamar,4 Orang,Ada ventilasi namun kadang terbuka/tertutup,Beresiko Gizi Lebih
67,Laki - laki,Kelurahan Batua,Kecamatan Manggala,Makassar,Ayah Karyawan Swasta,Ibu PNS,5.000.001 - 10.000.000,Tidak pernah diabetes,Tidak Telah BCG,Tidak pernah Opname,Tidak Ada,Tidak ASI Ekslusif,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak,Tidak Ada,Tidak ada yang mengkomsunsi obat TB dalam serumah,36 - 54 m^2,2 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,Beresiko Gizi Lebih
68

In [ ]:
cp["variabel"] = cp[['Jenis Kelamin', 'Alamat (Kelurahan)', 'Alamat (Kecamatan)',
       'Alamat (Kab/Kota)', 'Pekerjaan Ayah', 'Pekerjaan Ibu',
       'Pendapatan Orang Tua',
       'Apakah anak pernah mengalami penyakit diabetes?',
       'Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?',
       'Apakah anak pernah di opname sebelumnya?',
       'Jika pernah, anak diopname karena penyakit apa saja?',
       'Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan)',
       'Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?',
       'Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?',
       'Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang  serumah?',
       'Jika ada, penyakit apa saja?',
       'Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?',
       'Berapa luas rumah tempat anak tinggal?',
       'Berapa jumlah kamar tidur dalam rumah?',
       'Berapa jumlah orang yang tinggal dalam satu rumah?',
       'Bagaimana sistem ventilasi di rumah Anda? ', 'Status Gizi']].agg(','.join, axis=1)
       
print(cp['variabel'])

0     Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sor...
1     Perempuan,Kelurahan Parang Tambung,Kecamatan T...
2     Laki - laki,Kelurahan Kalebajeng,Kecamatan Baj...
3     Perempuan,Kelurahan Bone,Kecamatan Masamba,Luw...
4     Perempuan,Kelurahan Borong,Kecamatan Manggala,...
                            ...                        
66    Laki - laki,Kelurahan Tello Baru,Kecamatan Pan...
67    Laki - laki,Kelurahan Batua,Kecamatan Manggala...
68    Perempuan,Kelurahan Kapasa,Kecamatan Tamalanre...
69    Laki - laki,Kelurahan Kalabajeng,Kecamatan Baj...
70    Laki - laki,Kelurahan Maricaya,Kecamatan Makas...
Name: variabel, Length: 71, dtype: object


In [ ]:
cp_array = cp["variabel"].apply(lambda x: x.split(','))
print(cp_array.to_numpy)

<bound method IndexOpsMixin.to_numpy of 0     [Laki - laki, Kelurahan Noeha, Kecamatan Nuha,...
1     [Perempuan, Kelurahan Parang Tambung, Kecamata...
2     [Laki - laki, Kelurahan Kalebajeng, Kecamatan ...
3     [Perempuan, Kelurahan Bone, Kecamatan Masamba,...
4     [Perempuan, Kelurahan Borong, Kecamatan Mangga...
                            ...                        
66    [Laki - laki, Kelurahan Tello Baru, Kecamatan ...
67    [Laki - laki, Kelurahan Batua, Kecamatan Mangg...
68    [Perempuan, Kelurahan Kapasa, Kecamatan Tamala...
69    [Laki - laki, Kelurahan Kalabajeng, Kecamatan ...
70    [Laki - laki, Kelurahan Maricaya, Kecamatan Ma...
Name: variabel, Length: 71, dtype: object>


In [ ]:
# cp_str = cp_array.astype('str').apply(lambda x: x.split(','))
# cp_str

In [ ]:
test_data = [['I1','I2','I5'],
             ['I2','I4'],
             ['I2','I3'],
             ['I1','I2','I4'],
             ['I1','I3'],
             ['I2','I3'],
             ['I1','I3'],
             ['I1','I2','I3','I5'],
             ['I1','I2','I3']]


In [ ]:
from scipy.sparse import csr_matrix
from sklearn.base import BaseEstimator, TransformerMixin

class HotEncode(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None

    def fit(self, X):
        unique_items = set()
        for transaction in X:
            for item in transaction:
                unique_items.add(item)
        self.columns_ = sorted(unique_items)
        columns_mapping = {}
        for col_idx, item in enumerate(self.columns_):
            columns_mapping[item] = col_idx
        self.columns_mapping_ = columns_mapping
        return self

    def transform(self, X, sparse=False):
        if sparse:
            indptr = [0]
            indices = []
            for transaction in X:
                for item in set(transaction):
                    col_idx = self.columns_mapping_[item]
                    indices.append(col_idx)
                indptr.append(len(indices))
            non_sparse_values = [True]*len(indices)
            array = csr_matrix((non_sparse_values, indices, indptr),
                               dtype=bool)
        else:
            array = np.zeros((len(X), len(self.columns_)), dtype=bool)
            for row_idx, transaction in enumerate(X):
                for item in transaction:
                    col_idx = self.columns_mapping_[item]
                    array[row_idx, col_idx] = True
        return array

    def inverse_transform(self, array):
        return [[self.columns_[idx]
                 for idx, cell in enumerate(row) if cell]
                for row in array]

    def fit_transform(self, X, sparse=False):
        return self.fit(X).transform(X, sparse=sparse)

In [ ]:
pd.set_option('display.max_columns', None)
te = HotEncode()
te_ary = te.fit(cp_array).transform(cp_array)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Alergi,Asam Urat,Batuk,Bronkopneumonia,Demam,Diare,Eczema,Flu,Hipertensi,Kolesterol,Maag,Radang Tenggorokan,Sesak,Tuberkulosis,Tumor Payudara,1 Kamar,14 Orang,2 Kamar,2.000.000 - 5.000.000,2.500.000 - 5.000.000,3 Kamar,3 Orang,36 - 54 m^2,4 Kamar,4 Orang,5 Kamar,5 Orang,5.000.000 - 10.000.000,5.000.001 - 10.000.000,54 - 120 m^2,6 Kamar,6 Orang,7 Kamar,7 Orang,8 Orang,9 Orang,< 2.000.000,< 36 m^2,> 10.000.000,> 120 m^2,Ada ventilasi namun kadang terbuka/tertutup,Ada ventilasi namun selalu tertutup,Alergi Kulit,Asam Urat,Asma,Asma,Ayah ASN,Ayah Dokter,Ayah Dosen,Ayah Guru,Ayah Honorer,Ayah Karyawan Swasta,Ayah PNS,Ayah Pegawai Swasta,Ayah Pegawai kontrak RSUD,Ayah Pegawai swasta,Ayah Pelaut,Ayah Pensiunan,Ayah Perawat,Ayah TNI / POLRI,Ayah TNI-AD,Ayah Wiraswasta,Bantaeng,Barru,Batuk,Bekasi,Beresiko Gizi Lebih,Brongkhitis,Bronkhitis,Bronkitis,Bronkopneumonia,Demam,Demam Berdarah,Demam tinggi,Diabetes,Diabetes Tipe 1,Diare,Ginjal,Gizi Baik,Gizi Lebih,Gowa,Hanya ruangan depan/belakang,Hipertensi,Ibu ASN,Ibu Dokter,Ibu Dokter Gigi,Ibu Dosen,Ibu Guru,Ibu Ibu Rumah Tangga,Ibu Ibu Rumah tangga,Ibu Ibu rumah tangga,Ibu Karyawan Swasta,Ibu PNS,Ibu Pensiunan,Ibu Perawat,Ibu Sudah Alm,Ibu Swasta,Ibu Tidak Ada,Ibu Wiraswasta,Jantung,Kecamatan Bajeng,Kecamatan Bajeng Barat,Kecamatan Bantaeng,Kecamatan Barombong,Kecamatan Barru,Kecamatan Biring Kanaya,Kecamatan Biringkanaya,Kecamatan Bontoala,Kecamatan Bontomarannu,Kecamatan Bontonompo Selatan,Kecamatan Cikarang Selatan,Kecamatan Labakkang,Kecamatan Makassar,Kecamatan Mangarabombang,Kecamatan Manggala,Kecamatan Mariso,Kecamatan Marusu,Kecamatan Masamba,Kecamatan Minasatene,Kecamatan Moncongloe,Kecamatan Nuha,Kecamatan Pallangga,Kecamatan Panakkukang,Kecamatan Pangkajene,Kecamatan Pattallassang,Kecamatan Polewali,Kecamatan Rappocini,Kecamatan Sinjai Tengah,Kecamatan Somba Opu,Kecamatan Soreang,Kecamatan Tamalanrea,Kecamatan Tamalate,Kecamatan Ternate Selatan,Kecamatan Ujung Pandang,Kecamatan Ujung Tanah,Kelurahan Balang Baru,Kelurahan Bangkala,Kelurahan Bara-baraya,Kelurahan Barombong,Kelurahan Barua,Kelurahan Batua,Kelurahan Bitowa,Kelurahan Bone,Kelurahan Bonto-Bontoa,Kelurahan Bontolangkasa,Kelurahan Bontomanai,Kelurahan Borong,Kelurahan Bukit Harapan,Kelurahan Cikoang,Kelurahan Gunung Sari,Kelurahan Jongaya,Kelurahan Kalabajeng,Kelurahan Kalebajeng,Kelurahan Kalegowa,Kelurahan Kapasa,Kelurahan Karunrung,Kelurahan Labakkang,Kelurahan Malimongan Baru,Kelurahan Manggala,Kelurahan Mannuruki,Kelurahan Mappala,Kelurahan Maricaya,Kelurahan Mariso,Kelurahan Marumpa,Kelurahan Mataallo,Kelurahan Moncongloe,Kelurahan Moncongloe Lappara,Kelurahan Noeha,Kelurahan Pakatto,Kelurahan Palakka,Kelurahan Pallangga,Kelurahan Pallantikang,Kelurahan Panaikang,Kelurahan Pangkajene,Kelurahan Parang Tambung,Kelurahan Pattongko,Kelurahan Pisang Utara,Kelurahan Polewali,Kelurahan Romanglompoa,Kelurahan Romangpolong,Kelurahan Sengka,Kelurahan Serang,Kelurahan Sombalabella,Kelurahan Sudiang Raya,Kelurahan Tabaringan,Kelurahan Talasalapang,Kelurahan Tamalanrea,Kelurahan Tamalanrea Jaya,Kelurahan Tamamaung,Kelurahan Tamannyeleng,Kelurahan Tanah Tinggi,Kelurahan Tanah Tinggi Barat,Kelurahan Tello Baru,Kelurahan Tidung,Kelurahan Tombolo,Laki - laki,Luwu Utara,Makassar,Maluku Utara,Maros,Muntaber,Muntah,Obesitas,Pangkajene dan Kepulauan,Parepare,Perempuan,Pneumonia,Polewali Mandar,Radang Paru,Sesak,Sesak nafas,Setiap ruangan terdapat ventilasi,Sinjai,Sinusitis,Sorowako,Struma,Sydrom Nefrotik,Takalar,Tdk ada,Ternate,Tidak,Tidak ASI Ekslusif,Tidak Ada,Tidak Pernah,Tidak Telah BCG,Tidak ada,Tidak ada riwayat,Tidak ada riwayat penyakit diabetes (orang tua),Tidak ada yang mengkomsunsi obat TB dalam serumah,Tidak pernah,Tidak pernah Opname,Tidak pernah diabetes,Tipoid,Tonsilitis,Tuberkulosis,Ya,Ya ASI Ekslusif,Ya Telah BCG,Ya ada riwayat,Ya ada riwayat penyakit diabetes (orang tua),Ya ada yang mengkomsunsi obat TB dalam serumah,Ya pernah Opname,Ya pernah diabetes
0,False,False,False,False,False,False,False,False,False,False,Fal

# FP-Growth Algorithm

In [ ]:
import collections

def setup_fptree(df, min_support):
    num_itemsets = len(df.index)

    is_sparse = False
    if hasattr(df, "sparse"):
        if df.size == 0:
            itemsets = df.values
        else:
            itemsets = df.sparse.to_coo().tocsr()
            is_sparse = True
    else:
        itemsets = df.values

    item_support = np.array(np.sum(itemsets, axis=0) / float(num_itemsets))
    item_support = item_support.reshape(-1)

    items = np.nonzero(item_support >= min_support)[0]

    indices = item_support[items].argsort()
    rank = {item: i for i, item in enumerate(items[indices])}

    if is_sparse:
        itemsets.eliminate_zeros()
    tree = FPTree(rank)
    for i in range(num_itemsets):
        if is_sparse:
            nonnull = itemsets.indices[itemsets.indptr[i]:itemsets.indptr[i+1]]
        else:
            nonnull = np.where(itemsets[i, :])[0]
        itemset = [item for item in nonnull if item in rank]
        itemset.sort(key=rank.get, reverse=True)
        tree.insert_itemset(itemset)

    return tree, rank


def generate_itemsets(generator, num_itemsets, colname_map):
    itemsets = []
    supports = []
    for sup, iset in generator:
        itemsets.append(frozenset(iset))
        supports.append(sup / num_itemsets)

    res_df = pd.DataFrame({'support': supports, 'itemsets': itemsets})

    if colname_map is not None:
        res_df['itemsets'] = res_df['itemsets'] \
            .apply(lambda x: frozenset([colname_map[i] for i in x]))

    return res_df


def valid_input_check(df):
    if df.size == 0:
        return
    if hasattr(df, "sparse"):
        if not isinstance(df.columns[0], str) and df.columns[0] != 0:
            raise ValueError()

    # Fast path: if all columns are boolean, there is nothing to checks
    all_bools = df.dtypes.apply(pd.api.types.is_bool_dtype).all()
    if not all_bools:
        # Pandas is much slower than numpy, so use np.where on Numpy arrays
        if hasattr(df, "sparse"):
            if df.size == 0:
                values = df.values
            else:
                values = df.sparse.to_coo().tocoo().data
        else:
            values = df.values
        idxs = np.where((values != 1) & (values != 0))
        if len(idxs[0]) > 0:
            # idxs has 1 dimension with sparse data and 2 with dense data
            val = values[tuple(loc[0] for loc in idxs)]
            s = ((val))
            raise ValueError(s)


class FPTree(object):
    def __init__(self, rank=None):
        self.root = FPNode(None)
        self.nodes = collections.defaultdict(list)
        self.cond_items = []
        self.rank = rank

    def conditional_tree(self, cond_item, minsup):
        # Find all path from root node to nodes for item
        branches = []
        count = collections.defaultdict(int)
        for node in self.nodes[cond_item]:
            branch = node.itempath_from_root()
            branches.append(branch)
            for item in branch:
                count[item] += node.count

        # Define new ordering or deep trees may have combinatorially explosion
        items = [item for item in count if count[item] >= minsup]
        items.sort(key=count.get)
        rank = {item: i for i, item in enumerate(items)}

        # Create conditional tree
        cond_tree = FPTree(rank)
        for idx, branch in enumerate(branches):
            branch = sorted([i for i in branch if i in rank],
                            key=rank.get, reverse=True)
            cond_tree.insert_itemset(branch, self.nodes[cond_item][idx].count)
        cond_tree.cond_items = self.cond_items + [cond_item]

        return cond_tree

    def insert_itemset(self, itemset, count=1):
        self.root.count += count

        if len(itemset) == 0:
            return

        # Follow existing path in tree as long as possible
        index = 0
        node = self.root
        for item in itemset:
            if item in node.children:
                child = node.children[item]
                child.count += count
                node = child
                index += 1
            else:
                break

        # Insert any remaining items
        for item in itemset[index:]:
            child_node = FPNode(item, count, node)
            self.nodes[item].append(child_node)
            node = child_node

    def is_path(self):
        if len(self.root.children) > 1:
            return False
        for i in self.nodes:
            if len(self.nodes[i]) > 1 or len(self.nodes[i][0].children) > 1:
                return False
        return True

    def print_status(self, count, colnames):
        cond_items = [str(i) for i in self.cond_items]
        if colnames:
            cond_items = [str(colnames[i]) for i in self.cond_items]
        cond_items = ", ".join(cond_items)
        print((count, cond_items), end="\n")


class FPNode(object):
    def __init__(self, item, count=0, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = collections.defaultdict(FPNode)

        if parent is not None:
            parent.children[item] = self

    def itempath_from_root(self):
        path = []
        if self.item is None:
            return path

        node = self.parent
        while node.item is not None:
            path.append(node.item)
            node = node.parent

        path.reverse()
        return path

In [ ]:
import math
import itertools

def fpgrowth(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0):
    valid_input_check(df)

    if min_support <= 0.:
        raise ValueError(min_support)

    colname_map = None
    if use_colnames:
        colname_map = {idx: item for idx, item in enumerate(df.columns)}

    tree, _ = setup_fptree(df, min_support)
    minsup = math.ceil(min_support * len(df.index))  # min support as count
    generator = fpg_step(tree, minsup, colname_map, max_len, verbose)

    return generate_itemsets(generator, len(df.index), colname_map)


def fpg_step(tree, minsup, colnames, max_len, verbose):
    count = 0
    items = tree.nodes.keys()
    if tree.is_path():
        # If the tree is a path, we can combinatorally generate all
        # remaining itemsets without generating additional conditional trees
        size_remain = len(items) + 1
        if max_len:
            size_remain = max_len - len(tree.cond_items) + 1
        for i in range(1, size_remain):
            for itemset in itertools.combinations(items, i):
                count += 1
                support = min([tree.nodes[i][0].count for i in itemset])
                yield support, tree.cond_items + list(itemset)
    elif not max_len or max_len > len(tree.cond_items):
        for item in items:
            count += 1
            support = sum([node.count for node in tree.nodes[item]])
            yield support, tree.cond_items + [item]

    if verbose:
        tree.print_status(count, colnames)

    # Generate conditional trees to generate frequent itemsets one item larger
    if not tree.is_path() and (not max_len or max_len > len(tree.cond_items)):
        for item in items:
            cond_tree = tree.conditional_tree(item, minsup)
            for sup, iset in fpg_step(cond_tree, minsup,
                                      colnames, max_len, verbose):
                yield sup, iset

# Rules

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
rules = fpgrowth(df, min_support=0.4, use_colnames=True)
ruless = pd.DataFrame(rules)
ruless.sort_values(by=['support'], ascending=False)

,support,itemsets
0,0.985915,(Tidak pernah diabetes)
1,0.774648,(Tidak ada riwayat penyakit diabetes (orang tua))
16,0.760563,"(Tidak pernah diabetes, Tidak ada riwayat peny..."
2,0.746479,(Ya Telah BCG)
17,0.732394,"(Tidak pernah diabetes, Ya Telah BCG)"
3,0.718310,(Tidak ada yang mengkomsunsi obat TB dalam ser...
4,0.704225,(Tidak ada riwayat)
20,0.704225,"(Tidak pernah diabetes, Tidak ada yang mengkom..."
25,0.690141,"(Tidak pernah diabetes, Tidak ada riwayat)"
52,0.690141,"(Tidak pernah diabetes, Tidak Ada)"


Hasil analisis:
1. **98.2759%** anak yang tidak mengalami diabetes terkena **Tuberkulosis**
2. **82.7586%** anak yang telah menerima imunisasi BCG (*Bacillus Calmette-Guérin*) **terkena penyakit Tuberkulosis**
3. **81.0345%** anak yang tidak mengalami diabetes dan telah menerima imunasasi BCG (*Bacillus Calmette-Guérin*) **terkena penyakit Tuberkulosis**
